<a href="https://colab.research.google.com/github/TAbdullah-T/T5-SAD/blob/main/YOLO_Counting_Exiting_Cars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [2]:
# !pip install opencv
!pip install ultralytics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 528.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.0/872.0 kB 5.5 MB/s eta 0:00:00


In [7]:
import cv2
from ultralytics import YOLO, solutions

requirements: Ultralytics requirement ['shapely>=2.0.0'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.7 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 2.4s, installed 1 package: ['shapely>=2.0.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [8]:
model = YOLO('/content/YOLO_Model.pt')

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [47]:
video_path = '/content/YOLO_Video.mp4'
cap = cv2.VideoCapture(video_path)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [4]:
w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [48]:
video_writer = cv2.VideoWriter('output_video_12.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (int(w), int(h)))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [71]:
success, frame = cap.read()
import matplotlib.pyplot as plt
if success:
  plt.imshow(frame)

In [49]:

while cap.isOpened:
    success, frame = cap.read()
    if success:
      print('success')

      results = model.track(frame, persist=True)
      for count in counting_regions:
        x1, y1 , x2 , y2 = map(int, count['coord'])
        cv2.rectangle(frame, (int(x1),int(y1)),(int(x2),int(y2)), (0, 255, 0), 2)

    video_writer.write(frame)
    break
cap.release()
video_writer.release()
cv2.destroyAllWindows()

success

0: 384x640 2 buss, 56 cars, 1 truck, 2 vans, 284.4ms
Speed: 2.4ms preprocess, 284.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)


## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [8]:
model.names

NameError: name 'model' is not defined

In [72]:
#### github ultralytics/examples/YOLOv8-Region-Counter
counting_regions = [
    {
        "name": "Exit 1 ",
        'coord' :  [1060,230,  1130,300],
        "counts": 0,
    },
    {
        "name": "Exit 2 ",
        'coord' :  [600, 400,  650,550],
        "counts": 0,
    },
    {
        "name": "Exit 3 ",
        'coord' :  [690, 900,  770, 950],
        "counts": 0,
    },
    {
        "name": "Exit 4 ",
        'coord' :  [1270,590,   1350, 750],
        "counts": 0,
    },

]

In [65]:
def is_inside_region(point, region):
    (x, y) = point
    x1, y1, x2, y2 = map(int, region["coord"])
    return (x > x1 and x < x2 and y > y1 and y < y2)


In [ ]:
cap = cv2.VideoCapture(video_path)
video_writer = cv2.VideoWriter('output_video_6.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (int(w),int(h)))
import numpy as np

color = (0, 255, 0)
font = cv2.FONT_HERSHEY_SIMPLEX
i = 0
unique_car = []

while cap.isOpened():
  success, frame = cap.read()
  if not success:
    break

  results = model.track(frame, persist=True, )
  for box in results[0].boxes:

    bbox = box.xyxy[0].cpu().numpy()  # Bounding box coordinates
    class_id = int(box.cls[0].cpu().numpy()) if box.cls is not None else -1
    conf = box.conf[0].cpu().numpy() if box.conf is not None else 0.0
    id = int(box.id[0].cpu().numpy()) if box.id is not None else -1

    x1, y1, x2, y2 = map(int, bbox)
    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
    label = f'ID: {id} Class: {class_id} Conf: {conf:.2f}'
    cv2.putText(frame, label, (x1, y1 - 10), font, 0.5, color, 2, cv2.LINE_AA)

    bbox_center = (x1 + x2) // 2, (y1 + y2) // 2
    cv2.rectangle(frame,(bbox_center), (bbox_center), (225,255,255),5)

    for region in counting_regions:
      if is_inside_region(bbox_center, region):
        if not (id in unique_car):
          print(region['coord'], region['counts'],bbox_center )
          unique_car.append(id)
          region["counts"] += 1


  for region in counting_regions:
    region_label = region['name']+' count : ' + str(region["counts"])
    r_color = (0,0,0)
    text_color = (255,255,255)

    r_x1,r_y1,r_x2,r_y2 = region["coord"]

    cv2.putText(frame, region_label, (r_x1, r_y1+15), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
    cv2.rectangle(frame, (r_x1,r_y1), (r_x2,r_y2), color=(0,255,0), thickness= 2)

  cv2.putText(frame, 'Abdullah', (int(w)-200, int(h)-200), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 7)
  video_writer.write(frame)

  i+=1
  if i == 100:
    break

cap.release()
video_writer.release()


0: 384x640 2 buss, 59 cars, 2 trucks, 2 vans, 283.0ms
Speed: 2.4ms preprocess, 283.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 84 cars, 2 trucks, 3 vans, 267.0ms
Speed: 2.1ms preprocess, 267.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 85 cars, 2 trucks, 2 vans, 294.3ms
Speed: 2.1ms preprocess, 294.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 85 cars, 2 trucks, 2 vans, 268.9ms
Speed: 2.2ms preprocess, 268.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 85 cars, 2 trucks, 2 vans, 277.5ms
Speed: 2.1ms preprocess, 277.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 85 cars, 2 trucks, 2 vans, 278.6ms
Speed: 2.3ms preprocess, 278.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 85 cars, 2 trucks, 2 vans, 272.0ms
Speed: 2.1ms preprocess, 27

## another way

In [ ]:
for result in results[0]:
  for box in result.boxes:
    bbox = box.xyxy[0].cpu().tolist()
    cls

In [ ]:
cap_2 = cv2.VideoCapture(video_path)
video_writer2 = cv2.VideoWriter('Output_video_method2_1', cv2.VideoWriter_fourcc(*'mp4v'), fps, (int(w), int(h)))

In [58]:
regions = [
    {"name": "Exit 1", "coord": [1050,250, 1120,251], 'count': 0},
    {"name": "Exit 2", "coord": [600 ,450,  600,551], 'count': 0},
    {"name": "Exit 3", "coord": [680 ,900,  850,901], 'count': 0},
    {"name": "Exit 4", "coord": [1270,600, 1270,751], 'count': 0}]

i = 0
while cap_2.isOpened():
  success, frame = cap_2.read()
  if not success:
    break
  # Extract the results
  results = model.track(frame, persist=True)
  frame = results[0].plot()

  for result in results:
      for box in result.boxes:
          bbox = box.xyxy.cpu().tolist()
          x1, y1, x2, y2 = map(int, bbox)
          # cv2.rectangle(frame, (x1,y1),(x2,y2),)
          bbox_center = (x1 + x2) // 2, (y1 + y2) // 2  # center of each vehicle
          for region in regions:
              if is_inside_region(bbox_center, region):
                  counts[region["count"]] += 1
  offset = 300
  for region in regions:
    label = f'{region['name']} count : {region['count']}'
    cv2.putText(frame, (100, int(h)-offset))
    offset -= 50
    x1,y1,x2,y2 = region['coord']
    cv2.rectangle(frame, (x1,y1),(x2,y2), (0,255,0), 2)


  video_writer2(frame)
  if i == 100:
    break

cap_2.release()
video_writer2.release()

SyntaxError: f-string: unmatched '[' (<ipython-input-58-8ae5f70cd74e>, line 26)

## Save and Submit Your Work